# Chat Agent

This notebook goes through how to create a ChatGPT based agent

First, we set up the agent with tools as normal.

In [1]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain, LLMMathChain, SQLDatabase, SQLDatabaseChain

In [2]:
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
db = SQLDatabase.from_uri("sqlite:///../../../../notebooks/Chinook.db")
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="FooBar DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context"
    )
]

Now we create a ChatGPT based model and chain.

In [3]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.chains.llm import ChatModelChain
from langchain.agents.chat.base import ChatAgent

In [4]:
prompt = ChatAgent.create_prompt(tools)
llm_chain = ChatModelChain(llm=ChatOpenAI(temperature=0), prompt=prompt)

We can now use this as normal

In [5]:
tool_names = [tool.name for tool in tools]
agent = ChatAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [6]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.run("Who is Dua Lipa's boyfriend? What is his current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
Question: Who is Dua Lipa's boyfriend? What is his current age raised to the 0.43 power?
Thought: We need to find out the name of Dua Lipa's boyfriend and his current age.
Action: Search
Action Input: "Dua Lipa boyfriend"

Observation: Dua Lipa was seen heading home from the Saint Laurent Paris Fashion Week show with her new boyfriend Romain Gavras on Tuesday. Advertisement.
Thought:We found out the name of Dua Lipa's boyfriend, now we need to find out his age.
Action: Search
Action Input: "Romain Gavras age"

Observation: 41 years
Thought:We have the age of Romain Gavras, now we need to calculate his age raised to the 0.43 power.
Action: Calculator
Action Input: 41^(0.43)



> Entering new LLMMathChain chain...
41^(0.43)

```python
print(41**(0.43))
```

Answer: 4.9373857399466665

> Finished chain.

Observation: Answer: 4.9373857399466665

Thought:We have the answer to the second question, which is 4.9373857399466665.
Final Answer: Romain Gavra

"Romain Gavras is Dua Lipa's boyfriend and his current age raised to the 0.43 power is 4.9373857399466665."